In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from time import time
import os

# Problem 1
Assume that $X_1, X_2, ...$ is a sequence of iid Bernoulli random variables, and $X_i$ has probability $p$ of success.  Assume that $Y_1,Y_2, ...$ is another sequence of iid Bernoulli random variables, and every $Y_i$ has probability $q$ of success. Prove that if $p > q$, then $P\left(\frac{\sum_{i=1}^n X_i}{n} > \frac{1}{2}\right) > P\left(\frac{\sum_{i=1}^n Y_i}{n}  > \frac{1}{2}\right)$ for all $n$. Hint: write out a binomial expansion of the probabilities involved compare, term by term.

Note that since $p > q$, we can write $p = q+\delta$ for some $0 < \delta \leq 1-q$.  Let $Z_{i} \sim Bern\left(\frac{\delta}{1-q}\right)$, and $W_{i} = min\{1,Y_{i}+Z_{i}\}$.  Then $W_{i}=0$ with probability $(1-q)\left(1-\frac{\delta}{1-q}\right)$ because $Y_{i}$ and $Z_{i}$ are independent.  However, $(1-q)\left(1-\frac{\delta}{1-q}\right) = (1-q)\frac{1-q-\delta}{1-q} = 1-q-\delta = 1-(q+\delta) = 1-p$ is the probability that $X_{i} = 0$, and thus $W_{i} \sim X_{i}$.  Note that $X_{i} = W_{i} = 0$ implies $Y_{i} = 0$, but $Y_{i} = 0$ does not imply that $X_{i} = 0$ (because maybe $Z_{i} \not= 0$).  Thus $\{ i | Y_{i} > k\} \subset \{ i | X_{i} > k\}$ for all $k$.  Thus $P\left(\sum_{i=1}^{n} X_{i} > \frac{n}{2}\right) > P\left(\sum_{i=1}^{n} Y_{i} > \frac{n}{2}\right)$ for all $n$, which then gives $P\left(\frac{1}{n}\sum_{i=1}^{n} X_{i} > \frac{1}{2}\right) > P\left(\frac{1}{n}\sum_{i=1}^{n} Y_{i} > \frac{1}{2}\right)$.

# Problem 2
Assume that $X_i$ are iid Bernoulli random variables with probability $p ≥ \frac{2}{3}$ of success.  Use Cramer's theorem to give a lower bound on the number $n$ needed to give 95% confidence that $\sum_{i=1}^n \frac{X_i}{n} > \frac{1}{2}$. 

The worst $n$ will come from having the smallest probability of success, so to find a lower bound on the $\textit{needed}$ $n$, we can let $\mu = \frac{2}{3}$.  Cramer's Theorem states that $P\left(\left|\frac{1}{n}\sum_{i=1}^{n} X_{i} - \mu\right| \geq \epsilon \right) < 2e^{-2\epsilon^{2}n}$ for any $n$.  If we let $\epsilon = \frac{1}{6}$ and switch the inequality inside the probability to strictly less than, then that will guarantee that $\frac{1}{n} \sum_{i=1}^{n} X_{i} > \frac{1}{2}$, so we have $P\left(\frac{1}{n}\sum_{i=1}^{n} X_{i} > \frac{1}{2}\right) > P\left(\left|\frac{1}{n}\sum_{i=1}^{n} X_{i} - \frac{2}{3}\right| \leq \frac{1}{6} \right)= 1 - P\left(\left|\frac{1}{n}\sum_{i=1}^{n} X_{i} - \frac{2}{3}\right| \geq \frac{1}{6}\right) > 1 - 2e^{\frac{-2n}{36}} > 0.95 \Rightarrow 0.05 > 2e^{\frac{-2n}{36}} \Rightarrow ln\left(\frac{1}{40}\right) > -\frac{n}{18} \rightarrow n > 18ln(40) \approx 66.39983$, so the minimal $n$ is $n = 67$.

# Problem 3
Use scikit learn's random forest classifier to predict survival for the titanic dataset (use an 80-20 train-test split). Experiment with `n_estimators` in range$(20,201,20)$ and `max_depth` in range$(2,10)$ and compare training time and prediction accuracy.  Also compare to the results you obtained last time with a singe tree.

In [2]:
# Import dataset.
data = pd.read_csv('titanic.csv')
# Test train split.
x = data.filter(['Pclass','Sex','Age'])
x = x.fillna(0)
x = x.replace(to_replace='male',value=1)
x = x.replace(to_replace='female',value=0)
y = list(data['Survived'])
tr_x, ts_x, tr_y, ts_y = train_test_split(x,y,test_size=0.2)
combo = []
train_times = []
accs = []
# Experiment with max_depth and min_samples_leaf.
for max_depth in range(2,10) :
    for n_estimators in range(20,201,20) :
        combo.append((max_depth,n_estimators))
        clf = RandomForestClassifier(max_depth=max_depth,n_estimators=n_estimators)
        start = time()
        clf.fit(tr_x,tr_y)
        end = time()
        train_times.append(end-start)
        y_hat = clf.predict(ts_x)
        accuracy = sum([int(ts_y[i] == y_hat[i]) for i in range(len(y_hat))])/len(y_hat)
        accs.append(accuracy)

In [3]:
print('Depth\tNum Estimators\tTime\t\t\tAccuracy')
for i in range(len(combo)) :
    print(str(combo[i][0])+'\t'+str(combo[i][1])+'\t\t'+str(train_times[i])+'\t'+str(accs[i]))

Depth	Num Estimators	Time			Accuracy
2	20		0.10376787185668945	0.7541899441340782
2	40		0.06567215919494629	0.7653631284916201
2	60		0.08502602577209473	0.7374301675977654
2	80		0.12211394309997559	0.770949720670391
2	100		0.15097308158874512	0.7653631284916201
2	120		0.17398500442504883	0.7653631284916201
2	140		0.21190404891967773	0.7430167597765364
2	160		0.2398848533630371	0.7653631284916201
2	180		0.26813197135925293	0.7653631284916201
2	200		0.3300471305847168	0.7653631284916201
3	20		0.03299403190612793	0.7877094972067039
3	40		0.06352615356445312	0.7821229050279329
3	60		0.08589601516723633	0.7877094972067039
3	80		0.11332392692565918	0.7821229050279329
3	100		0.25925612449645996	0.7877094972067039
3	120		0.18869924545288086	0.770949720670391
3	140		0.23750972747802734	0.8044692737430168
3	160		0.39574289321899414	0.7821229050279329
3	180		0.4271259307861328	0.8044692737430168
3	200		0.32530713081359863	0.7821229050279329
4	20		0.033612966537475586	0.7988826815642458
4	40		0.06

# Problem 4
Do the same thing as #3, but on a large dataset related to your final project. 

In [4]:
path = '../../../../Senior Project/DATA/'

train = []
test = []

# Walk through player files
for dir_path , dir_name , file_names in os.walk(path) :
    # 2017 will be our testing set
    if '2017' in dir_path :
        for name in file_names :
            # Grab avgs file
            if name[-4:] == 'avgs' :
                data = pd.read_csv(os.path.join(dir_path,name))
                if isinstance(test,list) :
                    test = data.drop(['Unnamed: 0'],axis=1).as_matrix()
                else :
                    test = np.vstack((test,data.drop(['Unnamed: 0'],axis=1)))
    # Everything else will become our training set
    else :
        for name in file_names :
            # Grab avgs file
            if name[-4:] == 'avgs' :
                data = pd.read_csv(os.path.join(dir_path,name))
                if isinstance(train,list) :
                    train = data.drop(['Unnamed: 0'],axis=1).as_matrix()
                else :
                    train = np.vstack((train,data.drop(['Unnamed: 0'],axis=1).as_matrix()))

# From the way the data is saved, the last column is whether or not the player
#     is a score on how much of a contributor he was during the season.
train_x = train[:,:-1]
train_y = train[:,-1]
test_x = test[:,:-1]
test_y = test[:,-1]

In [6]:
combo = []
train_times = []
accs = []
# Experiment with max_depth and min_samples_leaf.
for max_depth in range(2,10) :
    for n_estimators in range(20,201,20) :
        combo.append((max_depth,min_samples_leaf))
        clf = RandomForestClassifier(max_depth=max_depth,n_estimators=n_estimators)
        start = time()
        clf.fit(train_x,train_y)
        end = time()
        train_times.append(end-start)
        y_hat = clf.predict(test_x)
        accuracy = sum([int(test_y[i] == y_hat[i]) for i in range(len(y_hat))])/len(y_hat)
        accs.append(accuracy)

In [7]:
print('Depth\tNum Estimators\tTime\t\t\tAccuracy')
for i in range(len(combo)) :
    print(str(combo[i][0])+'\t'+str(combo[i][1])+'\t\t'+str(train_times[i])+'\t'+str(accs[i]))

Depth	Num Estimators	Time			Accuracy
2	1		0.09115767478942871	0.8776041666666666
2	1		0.12742924690246582	0.8763020833333334
2	1		0.19207096099853516	0.8671875
2	1		0.2530171871185303	0.875
2	1		0.3210580348968506	0.875
2	1		0.3770761489868164	0.8736979166666666
2	1		0.4574589729309082	0.8736979166666666
2	1		0.5234029293060303	0.8723958333333334
2	1		0.6811022758483887	0.8736979166666666
2	1		0.6395609378814697	0.8736979166666666
3	1		0.0816347599029541	0.8671875
3	1		0.15685606002807617	0.8658854166666666
3	1		0.23860406875610352	0.87109375
3	1		0.3051891326904297	0.8723958333333334
3	1		0.38082313537597656	0.8723958333333334
3	1		0.44976115226745605	0.8723958333333334
3	1		0.5460841655731201	0.87109375
3	1		0.6161069869995117	0.8736979166666666
3	1		0.6863839626312256	0.8684895833333334
3	1		0.776176929473877	0.8736979166666666
4	1		0.10170698165893555	0.8763020833333334
4	1		0.19461488723754883	0.8723958333333334
4	1		0.292248010635376	0.87109375
4	1		0.3707289695739746	0.865885416